In [ ]:
# import settings and functions
%run ./../imports.ipynb

In [2]:
plot_base_2d = um_view + "/tutorials/fun-2/plot_base_2d"


Options
=======

[-load_file] if provided mesh is load from file

[-file_name] mesh name

[-base ainsworth, ainsworth_labatto, demkowicz, bernstein]

[-space h1, l2, hcurl, hdiv]

[-oder o] where is approximation order

In [3]:
base = 'demkowicz'
space = 'hcurl'
hdiv_transform = True
# hdiv_transform = False

# base = 'demkowicz'
# space = 'hdiv'

base = 'ainsworth'
space = 'h1'

# base = 'ainsworth'
# space = 'l2'

order = 3

In [ ]:
!rm out_*
!{plot_base_2d} \
-base {base} \
-space {space} \
-order {order} \
-log_sl verbose  

In [ ]:
!convert.py out*

In [ ]:
vtks = !ls -v out*vtk
print(vtks)

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1080, 1080))
display.start()

In [8]:
def show_glyphs(params, hdiv=False):
    out_to_vtk = !ls -c1 {params.show_file}*vtk
    last_file=out_to_vtk[0]

    p = pv.Plotter(notebook=True)

    mesh = pv.read(last_file[:-3] + "vtk")
    field_to_show = params.show_field

    # Subsample the mesh
    # mesh = mesh.extract_points(every_n=10)  # Keep every 10th point

    if hdiv:
        vectors = mesh.point_data[params.show_field]
        transformed_vectors = vectors.copy()
        transformed_vectors[:, 0] = vectors[:, 1]
        transformed_vectors[:, 1] = -vectors[:, 0]
        mesh.point_data["transformed_field"] = transformed_vectors
        field_to_show="transformed_field"

    # Add glyphs
    N = 10  # Change this to the desired value
    arrows = mesh.glyph(orient=field_to_show, scale=field_to_show, factor=0.3, tolerance=0.02)
    print(arrows.array_names)
    p.add_mesh(arrows, cmap=params.p_cmap, clim=params.clim, show_scalar_bar=params.show_scalar_bar)

    if params.show_edges:
        mesh=mesh.shrink(0.95)

    jupyter_backend='ipygany'

    p.add_mesh(mesh, show_scalar_bar=False, opacity=0.2, color='gray', smooth_shading=True)

    p.camera_position = params.camera_position
    print(f"Current camera position: {p.camera_position}")

    p.enable_parallel_projection()
    p.enable_image_style()

    p.show(jupyter_backend=jupyter_backend)
    if params.p_save:
        p.save_graphic(params.p_save)
        p.screenshot(params.p_save[:-3] + "png", window_size=[1080, 1080])

params.clim = [0,1]
params.p_cmap = "spectral"
params.show_field = "U"
params.show_file = vtks[0][:-3]
params.show_scalar_bar = False
params.show_edges = False
# show_glyphs(params)


In [9]:
params.show_ori_shape = True

In [ ]:
for i in range(0, len(vtks)):
    params.show_file = vtks[i][:-3]
    print(params.show_file)
    params.p_save = "shape_"+ str(i) +".pdf"
    print(space)
    params.camera_position =  [
            (0.5, 0.5, 3),
            (0.5, 0.5, 0),
            (0, 1, 0),
        ]
    # if space == 'hcurl' or space == 'hdiv':
    if space == 'hcurl':
        params.p_save = "shape_"+ space + "_" + str(i) +".pdf"
        if hdiv_transform:
            params.p_save = "shape_"+ "hdiv_" + str(i) +".pdf"
        show_glyphs(params, hdiv_transform)
    elif space == 'hdiv':
        params.p_save = "shape_"+ space + "_" + str(i) +".pdf"
        show_glyphs(params, hdiv=True)
    else: 
        params.show_field = "U"
        params.warp_field_scalar = "U" 
        params.field_part = -1
        # params.p_cmap = "PiYG"
        params.camera_position =  [
            (-2, -2, 3),
            (0.35, 0.35, -0.1),
            (0.25, 0.25, 1),
        ]

        zoom_factor = 0.4  # Adjust this value to your needs

        camera_position = params.camera_position

        # Calculate the new camera position
        new_camera_position = [
            camera_position[0][0] + (camera_position[1][0] - camera_position[0][0]) * zoom_factor,
            camera_position[0][1] + (camera_position[1][1] - camera_position[0][1]) * zoom_factor,
            camera_position[0][2] + (camera_position[1][2] - camera_position[0][2]) * zoom_factor,
        ]

        params.camera_position = [new_camera_position, camera_position[1], camera_position[2]]

        params.clim = [-1, 1]
        params.warp_factor = 0.5
        params.p_save = "shape_"+ space + "_" + str(i) +".pdf"
        show_results(params)
        params.p_save = "shape_"+ space + "_" + str(i) +".png"
        show_results(params)